In [1]:
from google.cloud import bigquery
import json
import os
import pandas as pd

GCP_PROJECT_NAME = 'opensource-observer'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../../oso_gcp_credentials.json'
bq_client = bigquery.Client(GCP_PROJECT_NAME)

In [2]:
from pyoso import Client
from dotenv import load_dotenv
load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
pyoso_client = Client(api_key=OSO_API_KEY)

# Part 1: Get applications

In [3]:
app_results = bq_client.query("""
    WITH submitted_apps AS (
      SELECT
        project_id,
        ARRAY_TO_STRING(ARRAY_AGG(DISTINCT round_id), ', ') AS round_ids
      FROM `op_atlas.application`
      WHERE
        status = 'submitted'
        AND round_id IN ('7', '8')
      GROUP BY 1
    ),
    
    chain_mappings AS (
      select distinct
        chain_id,
        CASE
          WHEN chain = 'op' THEN 'optimism'
          WHEN chain = 'fraxtal' THEN 'frax'
          ELSE chain
        END AS chain
      from `optimism_superchain_raw_onchain_data.transactions`
      where dt between '2025-02-24' and '2025-03-03'
    ),
    
    latest_project_info AS (
      SELECT
        app.project_id,
        app.round_ids,
        proj.open_source_observer_slug,
        MAX_BY(proj.name, proj.updated_at) AS name,
        MAX_BY(proj.description, proj.updated_at) AS description
      FROM `op_atlas.project` AS proj
      JOIN submitted_apps AS app
        ON app.project_id = proj.id
      GROUP BY app.project_id, app.round_ids, proj.open_source_observer_slug
    ),
    
    all_artifacts AS (
      SELECT DISTINCT *
      FROM (
    
        -- Blockchain artifacts
        SELECT
          lp.project_id,
          lp.name,
          lp.open_source_observer_slug,
          lp.round_ids,
          UPPER(chain_mappings.chain) AS artifact_source,
          LOWER(chain_mappings.chain) AS artifact_namespace,
          LOWER(pc.contract_address) AS artifact_name
        FROM latest_project_info lp
        JOIN `op_atlas.project_contract` pc
          ON lp.project_id = pc.project_id
        JOIN chain_mappings 
          ON CAST(pc.chain_id AS STRING) = CAST(chain_mappings.chain_id AS STRING)
    
        UNION ALL
    
        -- GitHub artifacts
        SELECT
          lp.project_id,
          lp.name,
          lp.open_source_observer_slug,
          lp.round_ids,
          'GITHUB' AS artifact_source,
          REGEXP_EXTRACT(LOWER(pr.url), r'github\.com/([^/]+)/') AS artifact_namespace,
          REGEXP_EXTRACT(LOWER(pr.url), r'github\.com/[^/]+/([^/]+)') AS artifact_name
        FROM latest_project_info lp
        JOIN `op_atlas.project_repository` pr ON lp.project_id = pr.project_id
        WHERE pr.`type` = 'github'
          AND pr.verified IS TRUE
    
        UNION ALL
    
        -- Website artifacts
        SELECT
          lp.project_id,
          lp.name,
          lp.open_source_observer_slug,
          lp.round_ids,
          'WEBSITE' AS artifact_source,
          REGEXP_EXTRACT(LOWER(pl.url), r'https?://(?:www\.)?([^/]+)') AS artifact_namespace,
          LOWER(pl.url) AS artifact_name
        FROM latest_project_info lp
        JOIN `op_atlas.project_links` pl ON lp.project_id = pl.project_id
      )
    )
    
    SELECT
      all_artifacts.*,
      abp.project_name AS oso_project_name
    FROM all_artifacts
    LEFT JOIN `metrics.artifacts_by_project_v1` abp
      ON
        all_artifacts.artifact_name = abp.artifact_name
        AND all_artifacts.artifact_namespace = abp.artifact_namespace
""")
df = app_results.to_dataframe()

In [4]:
def get_slugs(lst):
    s = list(set(x for x in lst if x and len(x) > 1))
    if not s:
        return None
    if len(s) == 1:
        return s[0]
    return " & ".join(s)

In [5]:
def stringify(lst):
    return "'" + "','".join(lst) + "'"

# Part 2: Review onchain builders

In [6]:
df8 = df[df['round_ids'].str.contains('8') == True].copy()

# df8_oso = df8.groupby(['project_id', 'name'], as_index=False).agg({
#     'open_source_observer_slug': get_slugs,
#     'oso_project_name': get_slugs
# })
# df8_oso = df8_oso[df8_oso['open_source_observer_slug'].isna()==False]
# df8_oso['valid'] = df8_oso['open_source_observer_slug'] == df8_oso['oso_project_name']
#df8_oso.to_json('data/onchain_builder_oso_mappings.json', indent=2, orient='records')

with open('data/onchain_builder_oso_mappings.json', 'r') as f:
    df8_mappings = {x['project_id']: x['open_source_observer_slug'] for x in json.load(f) if x['valid']}

df8['num_contract'] = df8['artifact_name'].apply(lambda x: int(x[:2] == '0x'))
df8['valid_oso_link'] = df8['project_id'].apply(lambda x: x in df8_mappings)
df8['valid_defillama_link'] = (df8['artifact_namespace'] == 'defillama.com')

df8_summary = (
    df8.groupby(['project_id', 'name']).agg({
        'num_contract': 'sum',
        'valid_oso_link': 'max',
        'valid_defillama_link': 'max',
    }).join(
        pd.read_csv('data/20250305_onchain_builders_eligibility.csv', index_col=0)
        .set_index(['project_id', 'name'])
    )
    .reset_index()
    .assign(is_eligible=lambda x: (
        (x['num_txns'] >= 1000) &
        (x['distinct_users'] >= 420) &
        (x['active_days'] >= 10)
    ))
    .sort_values(by=['is_eligible', 'num_txns'], ascending=False)
)
df8_summary['retrolist_url'] = df8_summary['project_id'].apply(lambda x: f"https://s7-onchain.retrolist.app/project/{x}")
df8_summary

,project_id,name,num_contract,valid_oso_link,valid_defillama_link,num_txns,distinct_users,active_days,gas_fees,chains,is_eligible,retrolist_url
38,0x4288aacdda65af69c03b2c9b9d057e2b93ed1b7671e4...,Aerodrome Finance,1,True,False,294932380.0,1405718.0,179.0,436765.125973,['BASE'],True,https://s7-onchain.retrolist.app/project/0x428...
75,0x780cf3d5aa4e3c94b8a4157f4b06a3f92ebcc2081358...,Virtuals Protocol,12,False,False,20926512.0,787209.0,179.0,6638.399360,['BASE'],True,https://s7-onchain.retrolist.app/project/0x780...
36,0x40ee6048c99aefd288f6133db207849d0951fe6b62af...,ORO,1,False,False,19360113.0,1111.0,117.0,6.523465,['WORLDCHAIN'],True,https://s7-onchain.retrolist.app/project/0x40e...
134,0xca3f26e1c876e821d2384a5cb699c6be494f9810301c...,BaseSwap,18,False,False,12202482.0,842647.0,179.0,869.111434,['BASE'],True,https://s7-onchain.retrolist.app/project/0xca3...
5,0x08df6e20a3cfabbaf8f34d4f4d048fe7da40447c24be...,Velodrome Finance,1,True,False,10997067.0,186863.0,181.0,520.061968,['MODE' 'OPTIMISM'],True,https://s7-onchain.retrolist.app/project/0x08d...
...,...,...,...,...,...,...,...,...,...,...,...,...
114,0xab21a3e2428a81b96eff96f345efe5e2673d8188c15d...,Fundit.Fun,3,False,False,NaN,NaN,NaN,NaN,NaN,False,https://s7-onchain.retrolist.app/project/0xab2...
116,0xafca3591b83586f14e21b397135b95af668f5ebb54cb...,Layerswap,0,True,True,NaN,NaN,NaN,NaN,NaN,False,https://s7-onchain.retrolist.app/project/0xafc...
124,0xbbb2ce15e5047267dcd0f25775dde250f27f9828a305...,Babylonians,0,False,False,NaN,NaN,NaN,NaN,NaN,False,https://s7-onchain.retrolist.app/project/0xbbb...
138,0xd3d00c946f62ac8cf49a85e3ab7d295d3b56f68d0b9f...,Compound Finance,3,False,False,NaN,NaN,NaN,NaN,NaN,False,https://s7-onchain.retrolist.app/project/0xd3d...


In [7]:
df8[df8['artifact_namespace'] == 'defillama.com']

,project_id,name,open_source_observer_slug,round_ids,artifact_source,artifact_namespace,artifact_name,oso_project_name,num_contract,valid_oso_link,valid_defillama_link
1399,0xafca3591b83586f14e21b397135b95af668f5ebb54cb...,Layerswap,Layerswap,8,WEBSITE,defillama.com,https://defillama.com/bridge/layerswap,None,0,True,True
2317,0x260127629089be48b8b1ee07c8962451a7f543741632...,BakerFi,,8,WEBSITE,defillama.com,https://defillama.com/protocol/bakerfi,None,0,False,True
2319,0x260127629089be48b8b1ee07c8962451a7f543741632...,BakerFi,None,8,WEBSITE,defillama.com,https://defillama.com/protocol/bakerfi,None,0,False,True
2635,0x692340d3ec51f53411126f8247003e2b87b113cfdf4a...,MemeWallet,None,8,WEBSITE,defillama.com,https://defillama.com/protocol/meme-wallet,None,0,False,True
2636,0x692340d3ec51f53411126f8247003e2b87b113cfdf4a...,MemeWallet,,8,WEBSITE,defillama.com,https://defillama.com/protocol/meme-wallet,None,0,False,True
3730,0x82edc3ac2a03da21c5a6033159ba8e97aab2f61dc5d1...,Superform,Optimism / Etherscan,8,WEBSITE,defillama.com,https://defillama.com/protocol/superform,None,0,False,True
4315,0xb26d9315cec30e60b52efd523c25a260c41d573458b2...,MintSwap Finance,,8,WEBSITE,defillama.com,https://defillama.com/protocol/mintswap-finance,None,0,False,True
4652,0x18923b60296e26a8ce35a0b4754cd6fd5427b2820a20...,Sake,,8,WEBSITE,defillama.com,https://defillama.com/protocol/sakefinance,None,0,False,True
7708,0xf9513b5bfca3be3513450bd56b48ae485ce157aa56c1...,LeverageX by Javsphere,,8,WEBSITE,defillama.com,https://defillama.com/protocol/javsphere,None,0,False,True
7709,0xf9513b5bfca3be3513450bd56b48ae485ce157aa56c1...,LeverageX by Javsphere,None,8,WEBSITE,defillama.com,https://defillama.com/protocol/javsphere,None,0,False,True


In [8]:
df8_summary.to_csv('data/20250305_onchain_builders_initial_eligibility_checks.csv', index=False)

# Part 3: Review devtools

In [9]:
df7 = df[df['round_ids'].str.contains('7') == True].copy()
df7.tail()

,project_id,name,open_source_observer_slug,round_ids,artifact_source,artifact_namespace,artifact_name,oso_project_name
63353,0xc6052138bbdae5976fa2866f46b0537182d4126c4bb9...,thirdweb,,7,BASE,base,0xa5855f36e032ee60a47b2ed603e5fb6b4c8160c4,third-web
63354,0xc6052138bbdae5976fa2866f46b0537182d4126c4bb9...,thirdweb,third-web,7,BASE,base,0xafb1e2b50fb5237ecca9b8c5bc5b6fa9b6aa9d4f,third-web
63355,0xc6052138bbdae5976fa2866f46b0537182d4126c4bb9...,thirdweb,,7,BASE,base,0x43a308f9b101847f2a317fc40d0b35550fa99dba,third-web
63356,0xc6052138bbdae5976fa2866f46b0537182d4126c4bb9...,thirdweb,,7,BASE,base,0xd19b1f98be299bdc6304d13c31edc833ef431703,third-web
63357,0xc6052138bbdae5976fa2866f46b0537182d4126c4bb9...,thirdweb,third-web,7,BASE,base,0x45d32634f2da957738bb6e427fabcf4bfd0d63ac,third-web


In [10]:
# df7_oso = df7.groupby(['project_id', 'name'], as_index=False).agg({
#     'open_source_observer_slug': get_slugs,
#     'oso_project_name': get_slugs
# })
# df7_oso = df7_oso[df7_oso['open_source_observer_slug'].isna()==False]
# df7_oso['valid'] = df7_oso['open_source_observer_slug'] == df7_oso['oso_project_name']
# df7_oso.to_json('data/devtooling_oso_mappings.json', indent=2, orient='records')

with open('data/devtooling_oso_mappings.json', 'r') as f:
    df7_mappings = {x['project_id']: x['open_source_observer_slug'] for x in json.load(f) if x['valid']}
df7_mappings# 

{'0x0008577196fa6ec286440b418e2f6305fc10e62ce759625d826be272ee6b45a3': 'createx',
 '0x08df6e20a3cfabbaf8f34d4f4d048fe7da40447c24be0f3ad513db6f13c755dd': 'velodrome',
 '0x1aacb85f1d87039696b876ddcae98c56d38ea9eed07b265409fdb256c8f8172a': 'blockhead-darryl-yeo',
 '0x229bbc8163bdee3891cde1c18c74ab3afc72e8683692fc7f1b767570cbe7c4a7': 'evm-explorer-pfed-prog',
 '0x45a05710ec665203dc05dfb3024e42532468fa6b75e6927c2da539d7fcac4f4f': 'openlabelsinitiative',
 '0x4dd10ba330a4a21459e2ba72f15fb5da2a63f7492e2b1567e6669c3d47fe6308': 'hyperledger-web3j',
 '0x50159544ec22884b777a997a625c75bb02e55b1dc8934aad3b78678e7c11d039': 'snekmate',
 '0x513265758a3804184b6a0771d99c1a4b04d3267bacd4b914e89ff991de229f6a': 'create2deployer',
 '0x517eaa9c56951de89261f2d7830ea49aae92f2a903104a17d9c5c2edd4959806': 'lifinance',
 '0x537097f27d55cb52d39b95c3ed65076349ab68aac48051d742d715456e0884d8': 'xdeployer',
 '0x62e37e96aa6e1cbfb6bd24b97c4b8f1e12cc3fe35d5388d2f041c42a12b40745': 'stargate-finance',
 '0x663e4d25ca3f3273652

In [11]:
devtooling_repos = bq_client.query("""

    WITH submitted_apps AS (
      SELECT project_id,
      FROM `op_atlas.application`
      WHERE
        status = 'submitted'
        AND round_id = '7'
    ),
    
    latest_project_info AS (
      SELECT
        app.project_id,
        MAX_BY(proj.name, proj.updated_at) AS name,
        MAX_BY(proj.description, proj.updated_at) AS description
      FROM `op_atlas.project` AS proj
      JOIN submitted_apps AS app
        ON app.project_id = proj.id
      GROUP BY app.project_id
    ),
    
    repos AS (
      SELECT
        lp.project_id,
        lp.name,
        REGEXP_EXTRACT(LOWER(pr.url), r'github\.com/([^/]+)/') AS artifact_namespace,
        REGEXP_EXTRACT(LOWER(pr.url), r'github\.com/[^/]+/([^/]+)') AS artifact_name
      FROM latest_project_info lp
      JOIN `op_atlas.project_repository` pr ON lp.project_id = pr.project_id
      WHERE pr.`type` = 'github'
        AND pr.verified IS TRUE
    )
    
    SELECT DISTINCT
      repos.project_id,
      repos.name,
      repos.artifact_namespace,
      repos.artifact_name,
      re.project_id AS oso_project_id,
      re.star_count,
      re.fork_count,
      re.num_packages_in_deps_dev,
      re.num_dependent_repos_in_oso
    FROM repos
    LEFT JOIN `oso.int_repositories_enriched` re
      ON re.artifact_namespace = repos.artifact_namespace
      AND re.artifact_name = repos.artifact_name

""")

df_devtooling_repos = devtooling_repos.to_dataframe()

In [12]:
df_devtooling_repos.tail()

,project_id,name,artifact_namespace,artifact_name,oso_project_id,star_count,fork_count,num_packages_in_deps_dev,num_dependent_repos_in_oso
347,0x8bfdc42f26bf691d378d2073ae509c46b85c0eed8db8...,Superfluid,must479,app,None,<NA>,<NA>,<NA>,<NA>
348,0x8bfdc42f26bf691d378d2073ae509c46b85c0eed8db8...,Superfluid,must479,splits-contracts,None,<NA>,<NA>,<NA>,<NA>
349,0x8bfdc42f26bf691d378d2073ae509c46b85c0eed8db8...,Superfluid,must479,simple-taiko-node,None,<NA>,<NA>,<NA>,<NA>
350,0x8bfdc42f26bf691d378d2073ae509c46b85c0eed8db8...,Superfluid,must479,account-abstraction,None,<NA>,<NA>,<NA>,<NA>
351,0x8bfdc42f26bf691d378d2073ae509c46b85c0eed8db8...,Superfluid,must479,ethereum-org-website,None,<NA>,<NA>,<NA>,<NA>


# Part 4. Add new projects / repos

In [13]:
df_new = pd.read_csv('data/20250303_project_repos_reviewed.csv').drop_duplicates()
df_github = df[df['artifact_source'] == 'GITHUB'].copy()

tests = ['0xaa1b878800206da24ee7297fb202ef98a6af0fb3ec298a65ba6b675cb4f4144b', 
         '0x4dd10ba330a4a21459e2ba72f15fb5da2a63f7492e2b1567e6669c3d47fe6308',
         '0x10da43868e7439419af2859f2539ed8b355a0f330bd05a6d028ddff8fd4a03d8']
df_github = df_github[~df_github['project_id'].isin(tests)]
df_github['github_url'] = df.apply(lambda x: f"https://github.com/{x['artifact_namespace']}/{x['artifact_name']}", axis=1)
df_github = df_github[['project_id', 'name', 'round_ids', 'github_url']].drop_duplicates(subset=['project_id', 'github_url'])
df_new = df_new.merge(df_github, on='github_url')
df_new['github_owner'] = df_new['github_url'].apply(lambda x: "/".join(x.split("/")[:-1]))
(
    df_new.query("is_available == True and is_fork == False")
    .groupby(['project_id', 'name', 'round_ids']).agg({
        'github_url': 'unique',
        'first_commit': 'min',
        'last_commit': 'max',
    })
    .sort_values(by='first_commit')
)#.to_csv("data/20250306_repos_to_add.csv")
#.to_json("data/20250306_repos_to_add.json")

,,,github_url,first_commit,last_commit
project_id,name,round_ids,,,
0xd63cb23c26cc3bda03b1cc2971589fd26c9d22f3bbc8f1ff2bb74613e6d1c36d,Solidity Bytes Arrays Utils Library,7,[https://github.com/gnsps/solidity-bytes-utils],2017-10-11T00:18:50Z,2025-02-06T01:00:17Z
0x670a2982f6d7786b58a8399bcabc23246365b21e533a97945e055791fc235e64,Hardhat,7,[https://github.com/nomicfoundation/edr],2018-04-14T22:20:28Z,2025-02-26T21:32:43Z
0x37fe5886f4c77d5a5cb947deff90158c045a5d207572763187748ac4dd4bd9b9,ethereum-multicall,7,[https://github.com/joshstevens19/ethereum-mul...,2020-10-04T08:09:52Z,2025-02-07T16:19:28Z
0x7974baaeef5e3af2e165a0911471ccddcb47d8ca74438558a89abb515aa64275,Krystal,8,[https://github.com/krystaldefi/krystal-core],2020-11-18T07:38:13Z,2025-02-26T04:50:31Z
0x7e917a2d0718401c9fe2a82f43ea558f5128f251b1e658c76dc7ff9a5e9fd993,PRBMath,7,[https://github.com/paulrberg/prb-math],2021-03-24T15:21:33Z,2025-02-22T13:03:26Z
...,...,...,...,...,...
0xb620f012eb197e5661f65c1a5aee6a4bbb43629f0056879ae4dc66adf8383ae3,ORB,8,[https://github.com/johncpalmer/orb_contracts],2025-02-19T23:04:48Z,2025-02-19T23:20:24Z
0x9e50828e261179c6d1a5c107c7c2c16907b4b529fef416f64af781c5b2ff29d1,Amazing Hunt,8,[https://github.com/amazinghunt-contracts/amaz...,2025-02-20T08:44:52Z,2025-02-20T09:07:43Z
0xa1f96576ad998e804140caccd478bae81b8b1059a107166df87b59fbf42c9e4a,OnChainGM,8,[https://github.com/onchaingm/onchaingm],2025-02-21T07:20:54Z,2025-03-01T12:33:06Z


In [14]:
from ossdirectory import fetch_data
from ossdirectory.fetch import OSSDirectory

ossd_data: OSSDirectory = fetch_data()
ossd_projects = ossd_data.projects

In [15]:
ossd_githubs = {}
for p in ossd_projects:
    for g in p.get('github',[]):
        ossd_githubs[g['url']] = p['name']

In [16]:
df_new

,github_url,is_available,is_fork,fork_of,first_commit,last_commit,total_commits,total_developers,project_id,name,round_ids,github_owner
0,https://github.com/meldsun0/samba,True,False,NaN,2024-08-06T02:41:07Z,2025-03-03T19:35:18Z,187,2,0x26a8d2bf9f36f9632cc4ae2157270087f3c0a0425f4f...,Samba,7,https://github.com/meldsun0
1,https://github.com/seetadev/zkmedical-billing,False,False,NaN,NaN,NaN,0,0,0x5cfdb8c453a7eede1946071e9d7417d01cc2116f5756...,Optimism × Filecoin: Enabling Scalable Storage...,7,https://github.com/seetadev
2,https://github.com/joshstevens19/ethereum-mult...,True,False,NaN,2020-10-04T08:09:52Z,2025-02-07T16:19:28Z,181,27,0x37fe5886f4c77d5a5cb947deff90158c045a5d207572...,ethereum-multicall,7,https://github.com/joshstevens19
3,https://github.com/collinsadi/diamonds,True,False,NaN,2024-10-18T16:19:34Z,2025-02-06T19:08:12Z,75,4,0x5f9f1b45eb4a49e8c2482288e02c8fc1e0e60ad3e88f...,Diamondscaffold: Simplifying EIP-2535 Diamond ...,7,https://github.com/collinsadi
4,https://github.com/mali030303/dragons-on-op-st...,True,False,NaN,2024-06-02T15:48:39Z,2024-06-02T15:54:17Z,3,1,0x6ee546715053790bda72f913925b782461830ccdcc14...,Dragons On Op Stack 🔴,7,https://github.com/mali030303
...,...,...,...,...,...,...,...,...,...,...,...,...
129,https://github.com/javsphere/contracts,True,False,NaN,2023-10-04T08:38:09Z,2025-02-19T15:12:29Z,33,2,0xf9513b5bfca3be3513450bd56b48ae485ce157aa56c1...,LeverageX by Javsphere,8,https://github.com/javsphere
130,https://github.com/javsphere/subgraph-leveragex,True,False,NaN,2024-07-10T08:29:01Z,2025-02-19T15:14:53Z,50,2,0xf9513b5bfca3be3513450bd56b48ae485ce157aa56c1...,LeverageX by Javsphere,8,https://github.com/javsphere
131,https://github.com/linkdrophq/linkdrop-p2p-sdk,True,False,NaN,2023-06-05T12:05:53Z,2024-12-20T13:40:21Z,362,2,0xd765664dd27255f1ba341fbb3375439a57fb6a92313d...,Linkdrop,8,https://github.com/linkdrophq
132,https://github.com/gallery-so/moshicam-contracts,True,False,NaN,2025-02-21T23:29:49Z,2025-02-21T23:45:22Z,3,1,0x0e5d90c5b5b75851d02f8286e78ed3b7af85d6920ca9...,Moshicam,8,https://github.com/gallery-so


In [17]:
df_new['in_oso'] = df_new.apply(
    lambda x: ossd_githubs.get(
        x['github_url'],
        ossd_githubs.get(
            x['github_owner']
        )        
    ), axis=1    
)

In [18]:
df_new[df_new['in_oso'].isna()==True].sort_values(by='name').to_csv("data/20250306_repos_to_add.csv")

# Sandbox

In [19]:
df[df['artifact_source'] == 'WORLDCHAIN']['name'].value_counts()

name
Morpho             28
MemeWallet         24
PUF                 7
DNA Token           7
zkCodex             2
World Billboard     2
InfoFinance         2
OnChainGM           2
CreateX             2
WorldPot            1
ORO                 1
Eat The Pie         1
ORB                 1
UNO                 1
Got Vision          1
Golden Otter        1
Eggs Vault          1
The Button          1
NFTs2Me             1
Name: count, dtype: int64

In [20]:
(
    df
    .query("artifact_source == 'GITHUB'")
    .groupby('artifact_namespace', as_index=False)
    .agg({'project_id': 'nunique', 'name': 'unique', 'oso_project_name': 'unique'})
    .sort_values(by='project_id', ascending=False)
)

,artifact_namespace,project_id,name,oso_project_name
51,ethereum,7,"[web3.py, Solidity, Remix Project, Sourcify, h...","[ethereum-miscellania, solidity-ethereum, web3..."
221,web3privacy,5,"[Newsletter, Jobs, Academy, Explorer, Usecase ...",[None]
141,partydao,4,"[Party.app + Party Protocol, ORO, UNO, create....",[party-dao]
144,pcaversaccio,4,"[xdeployer, Create2Deployer, CreateX, 🐍 snekmate]","[xdeployer, create2deployer, createx, snekmate]"
109,mali030303,4,"[Dragons On Op Stack 🔴, Skeleton ERA 💀, BASE B...",[None]
...,...,...,...,...
86,impersonator-eth,1,[Impersonator.xyz],[impersonator-eth]
87,infecteddotfun,1,[Infected],[None]
88,ionicprotocol,1,[Ionic Protocol],[ionicprotocol]
89,ipfs,1,[IPFS],[ipfs]
